In [1]:
import requests
import yfinance as yf
from bs4 import BeautifulSoup as bs
import pandas as pd
import datetime as dt
import pickle
import pprint as pp
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
def getSP500():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable', 'id': "constituents"})
    tickers_data = []
    for row in table.findAll('tr')[1:]:
        ticker_ = row.findAll('td')[0].text.rstrip()
        name_ = row.findAll('td')[1].text.rstrip()
        gics1_ = row.findAll('td')[3].text.rstrip()
        gics2_ = row.findAll('td')[4].text.rstrip()
        tickers_data.append([ticker_, name_, gics1_, gics2_])
    return tickers_data

In [6]:
def getNasdaq100():
    resp = requests.get('https://en.wikipedia.org/wiki/NASDAQ-100')
    soup = bs(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable', 'id': "constituents"})
    tickers_ = []
    for row in table.findAll('tr')[1:]:
        ticker_ = row.findAll('td')[1].text.rstrip()
        tickers_.append(ticker_)
    return tickers_

In [7]:
def getDow30():
    ticks = ['MMM', 'AXP', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'DIS', 'DOW', 'XOM', 'GS', 'HD', 'IBM', 'INTC', 
             'JNJ', 'JPM', 'MCD', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'RTX', 'TRV', 'UNH', 'VZ', 'V', 'WMT', 'WBA'] 
    return ticks

In [8]:
def getRussell2000():
    with open("../StockResearch/Russell2000.txt", "r") as f:
        content = f.readlines()
        content = [x.strip() for x in content]
        russell_ = []
        for row_, line_ in enumerate(content):
            if row_ % 2 == 0:
                russell_.append([line_])
            else:
                russell_[int(row_/2)].extend(line_.split("\t"))
        for stock in russell_:
            del(stock[4])
    return russell_

In [146]:
sp_ = getSP500()
naq_ = getNasdaq100()
dow_ = getDow30()
rus_ = getRussell2000()

In [109]:
tickers_sp = [ticker_[0] for ticker_ in sp_]
tickers_naq = naq_
tickers_dow = dow_
#tickers_rus = [ticker_[0] for ticker_ in rus_]

In [147]:
print(f'Nasdaq stocks not in the SP500:\n{set(tickers_naq) - set(tickers_sp)}')
print(f'Dow stocks not in the SP500:\n{set(tickers_dow) - set(tickers_sp)}')

Nasdaq stocks not in the SP500:
{'SGEN', 'TCOM', 'ASML', 'WDAY', 'SPLK', 'LBTYA', 'NXPI', 'ZM', 'SIRI', 'MELI', 'JD', 'CHKP', 'LBTYK', 'NTES', 'BIDU', 'LULU', 'TSLA', 'CSGP', 'BMRN'}
Dow stocks not in the SP500:
set()
[['AAN', "Aaron's Inc", 'Consumer Discretionary', 'Specialty Retail'], ['AAOI', 'Applied Optoelectronics Inc', 'Information Technology', 'Communications Equipment'], ['AAON', 'Aaon Inc', 'Industrials', 'Building Products'], ['AAT', 'American Assets Trust Inc', 'Real Estate', 'Equity Real Estate Investment Trusts (REITs)'], ['AAWW', 'Atlas Air Worldwide Holdings Inc', 'Industrials', 'Air Freight & Logistics']]


In [128]:
add_to_sp = {
 'ASML' : ['ASML Holding N.V.', 'Information Technology', 'Semiconductor Equipment'],
 'BIDU' : ['Baidu, Inc.', 'Communication Services', 'Interactive Media & Services'],
 'BMRN' : ['BioMarin Pharmaceutical Inc.', 'Health Care', 'Biotechnology'],
 'CHKP' : ['Check Point Software Technologies Ltd.', 'Information Technology', 'Software'],
 'CSGP' : ['CoStar Group, Inc.', 'Industrials', 'Professional Services'],
 'JD'   : ['JD.com, Inc.', 'Consumer Discretionary', 'Internet & Direct Marketing Retail'],
 'LBTYA': ['Liberty Global plc', 'Communication Services', 'Diversified Telecommunication Services'],
 'LBTYK': ['Liberty Global plc', 'Communication Services', 'Diversified Telecommunication Services'],
 'LULU' : ['Lululemon Athletica Inc.', 'Consumer Discretionary', 'Textiles, Apparel & Luxury Goods'],
 'MELI' : ['MercadoLibre, Inc.', 'Consumer Discretionary', 'Internet & Direct Marketing Retail'],
 'NTES' : ['NetEase, Inc.', 'Communication Services', 'Entertainment'],
 'NXPI' : ['NXP Semiconductors N.V.', 'Information Technology', 'Semiconductor Equipment'],
 'SGEN' : ['Seattle Genetics, Inc.', 'Health Care', 'Biotechnology'],
 'SIRI' : ['Sirius XM Holdings Inc.', 'Communication Services', 'Media'],
 'SPLK' : ['Splunk Inc.', 'Information Technology', 'Software'],
 'TSLA' : ['Tesla, Inc.', 'Consumer Discretionary', 'Automobiles'],
 'TCOM' : ['Trip.com', 'Consumer Discretionary', 'Internet & Direct Marketing Retail'],
 'WDAY' : ['Workday, Inc.', 'Information Technology', 'Software'],
 'ZM'   : ['Zoom Video Communications, Inc.', 'Information Technology', 'Software']}

In [161]:
sp_df = pd.DataFrame(sp_, columns = ["Ticker", "Name", "GICS-1", "GICS-2"]).set_index('Ticker')
rus_df = pd.DataFrame(rus_, columns = ["Ticker", "Name", "GICS-1", "GICS-2"]).set_index('Ticker')
add_to_sp_df = pd.DataFrame.from_dict(add_to_sp, orient = 'index', columns = ['Name', 'GICS-1', 'GICS-2'])

total_df = sp_df.append(rus_df)
total_df = total_df.append(add_to_sp_df)
total_df

,Name,GICS-1,GICS-2
MMM,3M Company,Industrials,Industrial Conglomerates
ABT,Abbott Laboratories,Health Care,Health Care Equipment
ABBV,AbbVie Inc.,Health Care,Pharmaceuticals
ABMD,ABIOMED Inc,Health Care,Health Care Equipment
ACN,Accenture plc,Information Technology,IT Consulting & Other Services
...,...,...,...
SPLK,Splunk Inc.,Information Technology,Software
TSLA,"Tesla, Inc.",Consumer Discretionary,Automobiles
TCOM,Trip.com,Consumer Discretionary,Internet & Direct Marketing Retail
WDAY,"Workday, Inc.",Information Technology,Software


In [162]:
naq_df = total_df.loc[tickers_naq]
dow_df = total_df.loc[tickers_dow]
with open("IndexComponents.pickle","wb") as f:
    pickle.dump([sp_df, naq_df, dow_df, rus_df], f)

In [2]:
with open("IndexComponents.pickle","rb") as f:
    [sp_df, naq_df, dow_df, rus_df] = pickle.load(f)

In [13]:
writer_ =  pd.ExcelWriter("IndexComponents" + '.xlsx', engine = 'xlsxwriter')
sp_df.to_excel(writer_, "S&P500", index = True)
naq_df.to_excel(writer_, "NQ100", index = True)
dow_df.to_excel(writer_, "Dow30", index = True)
rus_df.to_excel(writer_, "Russell2000", index = True)
writer_.save()